In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [5]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
x = loadmat('/DatosEntrenamiento2.mat')

##Traernos los datos del archivo .mat que nos interesan
y = x['data_te']

##Convertir los datos en un array de Numpy
z = np.array(y)

##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)

##Contar el numero de datos nulos del dataFrame
np.count_nonzero(pd.isnull(df) == True)

df.columns = ['uplink_size_mean','uplink_size_median','uplink_size_MAD','uplink_size_STD','uplink_size_Skewness','uplink_size_Kurtosis','uplink_size_MAX','uplink_size_MIN','uplink_size_MeanSquare','downlink_size_mean','downlink_size_median','downlink_size_MAD', 'downlink_size_STD','downlink_size_Skewness','downlink_size_Kurtosis','downlink_size_MAX','downlink_size_MIN','downlink_size_MeanSquare','both_links_size_mean','both_links_size_mean','both_links_size_MAD','both_links_size_STD','both_links_size_Skewness','both_links_size_Kurtosis','both_links_size_MAX','both_links_size_MIN','both_links_size_MeanSquare','uplink_interval_mean','uplink_interval_median','uplink_interval_MAD','uplink_interval_STD','uplink_interval_Skewness','uplink_interval_Kurtosis','uplink_interval_MAX','uplink_interval_MIN','uplink_interval_MeanSquare','downlink_interval_mean','downlink_interval_median','downlink_interval_MAD','downlink_interval_STD','downlink_interval_Skewness','downlink_interval_Kurtosis','downlink_interval_MAX','downlink_interval_MIN','downlink_interval_MeanSquare','both_links_interval_mean','both_links_interval_median','both_links_interval_MAD','both_links_interval_STD','both_links_interval_Skewness','both_links_interval_Kurtosis','both_links_interval_MAX','both_links_interval_MIN','both_links_interval_MeanSquare','Label']
df.sample(8)

,uplink_size_mean,uplink_size_median,uplink_size_MAD,uplink_size_STD,uplink_size_Skewness,uplink_size_Kurtosis,uplink_size_MAX,uplink_size_MIN,uplink_size_MeanSquare,downlink_size_mean,downlink_size_median,downlink_size_MAD,downlink_size_STD,downlink_size_Skewness,downlink_size_Kurtosis,downlink_size_MAX,downlink_size_MIN,downlink_size_MeanSquare,both_links_size_mean,both_links_size_mean,both_links_size_MAD,both_links_size_STD,both_links_size_Skewness,both_links_size_Kurtosis,both_links_size_MAX,both_links_size_MIN,both_links_size_MeanSquare,uplink_interval_mean,uplink_interval_median,uplink_interval_MAD,uplink_interval_STD,uplink_interval_Skewness,uplink_interval_Kurtosis,uplink_interval_MAX,uplink_interval_MIN,uplink_interval_MeanSquare,downlink_interval_mean,downlink_interval_median,downlink_interval_MAD,downlink_interval_STD,downlink_interval_Skewness,downlink_interval_Kurtosis,downlink_interval_MAX,downlink_interval_MIN,downlink_interval_MeanSquare,both_links_interval_mean,both_links_interval_median,both_links_interval_MAD,both_links_interval_STD,both_links_interval_Skewness,both_links_interval_Kurtosis,both_links_interval_MAX,both_links_interval_MIN,both_links_interval_MeanSquare,Label
8622,3.419266,23.925880,0.000580,0.000673,6.577259,42.121199,167.482440,0.000079,23.926063,265.326531,58.981066,228.0,0.0000,1.540397,1.342768,422.0,228.0,271.672484,3.285161,23.419657,0.000282,0.000285,6.726817,44.116326,167.255412,0.000090,23.420463,269.960784,82.091159,228.0,0.0000,1.920765,2.122389,486.0,228.0,281.931990,1.675442,16.725228,0.000209,0.000127,9.701975,93.059371,167.255062,0.000079,16.725521,267.69,71.383542,228.0,0.0000,1.946161,2.700305,486.0,228.0,276.952324,0.0
4885,0.012026,0.014478,0.004125,0.006108,1.084265,3.259433,0.049661,0.000005,0.018526,118.052632,41.010126,88.0,0.0000,0.566301,1.355949,183.0,86.0,124.618365,0.002846,0.009169,0.000089,0.000121,4.613221,27.056476,0.063629,0.000006,0.009546,957.530864,637.189220,1386.0,124.5384,-0.621789,1.424561,1478.0,76.0,1147.982137,0.002305,0.007412,0.000036,0.000043,4.243428,22.414728,0.049661,0.000005,0.007727,798.03,661.775839,1378.0,148.2600,-0.103436,1.039973,1478.0,76.0,1034.610879,1.0
3903,0.014074,0.032515,0.001523,0.002249,2.762050,9.100492,0.116919,0.000005,0.034803,127.791667,41.338454,128.0,59.3040,-0.000176,1.001995,171.0,86.0,134.046167,0.004567,0.020492,0.000022,0.000022,5.518520,32.683198,0.126843,0.000006,0.020863,1026.118421,543.184437,1346.0,118.6080,-1.046556,2.273198,1522.0,76.0,1159.347281,0.003471,0.016786,0.000022,0.000021,6.302740,42.607044,0.116919,0.000005,0.017059,810.52,610.410683,1220.0,418.0932,-0.229437,1.141319,1522.0,76.0,1012.826668,1.0
11936,0.667932,1.978708,0.011627,0.017238,3.996878,16.320470,10.840400,0.000120,2.064226,359.589700,244.022100,280.0,0.0000,3.040043,8.296142,1368.0,280.0,432.810300,0.431399,1.593310,0.000354,0.000262,5.141187,27.607660,10.639090,0.000125,1.638025,1239.607000,501.803500,1536.0,0.0000,-1.175363,-0.461392,1536.0,280.0,1335.778000,0.263153,1.234962,0.000423,0.000322,6.982309,51.629510,10.639090,0.000120,1.256634,896.40,601.302200,784.0,747.2304,0.046377,-1.935229,1536.0,280.0,1077.721000,0.0
11417,0.002530,0.002455,0.002072,0.002747,1.115917,1.250080,0.010925,0.000115,0.003506,217.333300,3.813850,216.0,0.0000,2.392838,3.811852,228.0,216.0,217.366100,0.002084,0.002213,0.001458,0.001555,2.914175,12.177300,0.013873,0.000336,0.003025,1622.945000,245.441000,1676.0,0.0000,-4.705216,20.842600,1676.0,381.0,1641.066000,0.001148,0.001384,0.000750,0.000761,3.847043,22.645830,0.010925,0.000115,0.001793,990.42,725.811100,1676.0,0.0000,-0.118432,-1.999457,1676.0,216.0,1225.751000,0.0
7530,0.005352,0.008223,0.000662,0.000951,1.243730,2.880838,0.022100,0.000012,0.009692,111.034483,47.258017,88.0,17.7912,1.034370,2.232205,198.0,76.0,120.353502,0.002357,0.005569,0.000260,0.000360,2.615536,8.698062,0.022046,0.000006,0.006011,920.478873,742.041987,1506.0,154.1904,-0.215879,1.102204,1610.0,76.0,1179.047228,0.001673,0.004788,0.00

In [6]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import itertools as itertools

In [8]:
def experiementarSVC(x, y, kernels, gammas,params_reg):
    """función que realizar experimentos sobre un SVM para clasificación
    
    x: numpy.Array, con las caracteristicas del problema
    y: numpy.Array, con la variable objetivo
    kernels: List[str], lista con valores a pasar 
        a sklearn correspondiente al kernel de la SVM
    gammas: List[float], lista con los valores a pasar a
        sklean correspondiente el valor de los coeficientes para usar en el
        kernel
    params_reg: List[float], lista con los valores a a pasar a 
        sklearn para ser usados como parametro de regularización
    
    retorna: pd.Dataframe con las siguientes columnas:
        - 3 columnas con los tres parametros: kernel, gamma, param de regularizacion
        - error cuadratico medio en el cojunto entrenamiento (promedio de los 4 folds)
        - error cuadratico medio en el cojunto test (promedio de los 4 folds)
        - % de Vectores de Soporte promedio para los 4 folds (0 a 100)
    """
    idx = 0
    kf = StratifiedKFold(n_splits=4)
    # crear una lista con la combinaciones de los elementos de cada list
    kernels_gammas_regs = list(itertools.product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) # puede usar para ver los params
        errores_train = []
        errores_test = []
        pct_support_vectors = []        
        for train_index, test_index in kf.split(x, y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  
            # normalizar los datos
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            svm = SVC(kernel=kernel, gamma=gamma, C=param_reg, max_iter = 100)
            # Entrenar el modelo
            svm.fit(X=X_train, y=y_train)
            # calculo de errores
            y_train_pred = svm.predict(X=X_train)
            y_test_pred = svm.predict(X=X_test)
            # error y pct de vectores de soporte
            errores_train.append(accuracy_score(y_true = y_train, y_pred =y_train_pred))
            errores_test.append(accuracy_score(y_true = y_test, y_pred = y_test_pred))
            # contar muestras de entrenamiento
            n_train = X_train.shape[0]
            pct_vs = (len(svm.support_vectors_)/n_train)*100
            pct_support_vectors.append(pct_vs)
        
        resultados.loc[idx,'kernel'] = kernel
        resultados.loc[idx,'gamma'] = gamma
        resultados.loc[idx,'param_reg'] = param_reg
        resultados.loc[idx,'error de entrenamiento'] = np.mean(errores_train)
        resultados.loc[idx,'error de prueba'] = np.mean(errores_test)
        resultados.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
        idx+=1
    return (resultados)

In [9]:
resultadosSVC = experiementarSVC(x = XEntrenamiento,y=YEntrenamiento,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.001, 0.01,0.1, 1.0,10]
                                )

resultadosSVC

parametros usados ('linear', 0.01, 0.001)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('linear', 0.01, 0.01)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('linear', 0.01, 0.1)


...

parametros usados ('rbf', 0.1, 1.0)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('rbf', 0.1, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


,kernel,gamma,param_reg,error de entrenamiento,error de prueba,% de vectores de soporte
0,linear,0.01,0.001,1.0,1.000000,1.699921
1,linear,0.01,0.010,1.0,1.000000,0.648095
2,linear,0.01,0.100,1.0,1.000000,0.231613
3,linear,0.01,1.000,1.0,1.000000,0.220989
4,linear,0.01,10.000,1.0,1.000000,0.220989
5,linear,0.10,0.001,1.0,1.000000,1.699921
6,linear,0.10,0.010,1.0,1.000000,0.648095
7,linear,0.10,0.100,1.0,1.000000,0.231613
8,linear,0.10,1.000,1.0,1.000000,0.220989
9,linear,0.10,10.000,1.0,1.000000,0.220989
